In [ ]:

# Classificação com KNN (k=3) usando distância de Manhattan + visualização

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Dados dos clientes de treino

clientes = pd.DataFrame({
    "Idade": [25, 35, 45, 30],
    "Renda": [50, 65, 85, 70],
    "Compras": [20, 40, 15, 30],
    "Classe": ["Baixo", "Médio", "Baixo", "Médio"]
}, index=["A", "B", "C", "D"])

# Novo cliente E
cliente_E = pd.DataFrame({
    "Idade": [40],
    "Renda": [95],
    "Compras": [60]
}, index=["E"])


# Normalização

scaler = MinMaxScaler()
X_train = scaler.fit_transform(clientes[["Idade", "Renda", "Compras"]])
X_E = scaler.transform(cliente_E)


# Codificação das classes

encoder = LabelEncoder()
y_train = encoder.fit_transform(clientes["Classe"])


# Cálculo manual da distância de Manhattan

dist_manhattan = np.sum(np.abs(X_train - X_E), axis=1)
idx_vizinhos = np.argsort(dist_manhattan)[:3]
classes_vizinhos = clientes.iloc[idx_vizinhos]["Classe"].values
distancias_vizinhos = dist_manhattan[idx_vizinhos]


# Modelo KNN

knn = KNeighborsClassifier(n_neighbors=3, metric='manhattan')
knn.fit(X_train, y_train)
pred_codificada = knn.predict(X_E)
classe_prevista = encoder.inverse_transform(pred_codificada)


# Resultados

print("Classe prevista para o Cliente E:", classe_prevista[0])
print("Vizinhos mais próximos:", classes_vizinhos)
print("Distâncias aos vizinhos:", distancias_vizinhos)


# Visualização

cores = {"Baixo": "blue", "Médio": "orange", "Alto": "green"}

fig, ax = plt.subplots()

# Pontos dos clientes anteriores
for classe in clientes["Classe"].unique():
    grupo = clientes[clientes["Classe"] == classe]
    ax.scatter(grupo["Idade"], grupo["Compras"],
               label=f"{classe}", color=cores[classe], s=80)

# Cliente E
ax.scatter(cliente_E["Idade"], cliente_E["Compras"],
           label=f"Cliente E ({classe_prevista[0]})", color="red", marker="X", s=120, edgecolors="black")

# Linhas para os vizinhos
for idx in idx_vizinhos:
    viz = clientes.iloc[idx]
    ax.plot([viz["Idade"], cliente_E.iloc[0]["Idade"]],
            [viz["Compras"], cliente_E.iloc[0]["Compras"]],
            linestyle='dashed', color='gray')

ax.set_title("Classificação KNN - Cliente E")
ax.set_xlabel("Idade")
ax.set_ylabel("Compras no último ano")
ax.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
